In [1]:
import os
import math
import numpy as np

import cv2
from matplotlib import pyplot as plt

In [5]:
book_dir = 'kokuyaku1'
filename = 'kokuyakukanbunta058800uoft_0400.jp2'

path = os.path.join(book_dir, filename)

In [6]:

img = cv2.imread(path)
height, width, channels = img.shape

# Grayscale processing
gray = cv2.cvtColor(img, code=cv2.COLOR_BGR2GRAY)
alpha = 1.5
beta = -50
contrast = cv2.convertScaleAbs(gray, alpha=alpha, beta=beta)

retval, img2 = cv2.threshold(contrast, thresh=180, maxval=255, type=cv2.THRESH_TOZERO)

# Detect lines
edges = cv2.Canny(img2, threshold1=10, threshold2=200)

contours, hierarchy = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

line_coords = []
line_contours = []
for cnt in contours:
    arclen = cv2.arcLength(cnt, True)
    approx = cv2.approxPolyDP(cnt, arclen*0.01, True)
    #drawContours
    if len(approx) == 2:
        min_diff, max_diff = np.sort(np.ptp(approx[:,0], axis=0))
        if min_diff < 10 and max_diff > 40:
            line_contours.append(approx)
            line_coords.extend(approx)

line_coords = np.reshape(line_coords, (-1, 2))

# Remove lines in upper quarter of image (underline for running header)
line_coords = line_coords[line_coords[:,1]>height//4]

new_contours = np.array([np.min(line_coords, axis=0), np.max(line_coords, axis=0)])
new_contours

array([[ 567, 1660],
       [1459, 2944]], dtype=int32)

In [15]:
cv2.rectangle(img2, *new_contours, (255, 255, 255), -1)
cv2.imwrite("output.jpg", img2)

True

In [14]:
tuple(img[20][height//2])

(180, 223, 234)